<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M1-airbnb-eda-geoplot-starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs
!pip install pydeck -q
!pip install folium

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns #seaborn plots
from matplotlib import pyplot as plt #plot control
sns.set() #plot style

#Geoplotting with folium/leaflet
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

#Fancy geoplotting with DeckGL
import pydeck as pdk

In [ ]:
data = pd.read_csv('http://data.insideairbnb.com/denmark/hovedstaden/copenhagen/2022-06-24/visualisations/listings.csv')

In [ ]:
data

In [ ]:
data = data[data.number_of_reviews > 0]

In [ ]:
data.room_type.value_counts()

In [ ]:
data.room_type.unique()

In [ ]:
data = data[data.room_type.isin(['Private room', 'Entire home/apt'])]

In [ ]:
data.neighbourhood.value_counts()

In [ ]:
palette=['#FF8C00','#A034F0', '#159090']

In [ ]:
plt.figure(figsize=(10,6))

sns.boxplot(data = data, x = "price", y = "neighbourhood" , 
            hue = "room_type", palette=palette)

In [ ]:
data['price_z'] = (data['price'] -data['price'].mean())/data['price'].std(ddof=0)

In [ ]:
data['price_z'] = data['price_z'].abs()

In [ ]:
data = data[data.price_z < 3]

In [ ]:
plt.figure(figsize=(10,6))

sns.boxplot(data = data, x = "price", y = "neighbourhood" , 
            hue = "room_type", palette=palette)

In [ ]:
plt.figure(figsize=(10,6))

sns.boxplot(data = data, x = "price", y = "neighbourhood" , 
            hue = "room_type", palette=palette, showfliers=False)

In [ ]:
data['log_price'] = np.log(data['price'])

In [ ]:
g = sns.lmplot(x="reviews_per_month",
               y="log_price",
               hue="room_type",
               height=7,
               data=data,
                scatter_kws={'alpha':0.5},
               palette=palette)
g.set_xlabels('Reviews/month')
g.set_ylabels('Price')

In [ ]:
data_s = data.sample(1000)

In [ ]:
#Define the map
map = folium.Map(location=[55.7800, 12.5400], zoom_start=12)

#add prettier base-layer
folium.TileLayer('cartodbpositron').add_to(map)

#add points (in a loop)
for j in data_s.iterrows():
    marker= folium.Circle(
        radius=30,
        location=[j[1]['latitude'], j[1]['longitude']],
        tooltip=j[1]['name'],
        popup=f"<b>{j[1]['name']}</b>\n {j[1]['price']}",
        color='purple',
        fill=True
    ).add_to(map)

#locations = zip(data['longitude'], data['latitude'])
#HeatMap(locations).add_to(map)

In [ ]:
map

In [ ]:
#Define the map
map = folium.Map(location=[55.7800, 12.5400], zoom_start=12)

#add prettier base-layer
folium.TileLayer('cartodbpositron').add_to(map)

#add points (in a loop)
for j in data_s.iterrows():
    marker= folium.Circle(
        radius=30,
        location=[j[1]['latitude'], j[1]['longitude']],
        tooltip=j[1]['name'],
        popup=f"<b>{j[1]['name']}</b>\n {j[1]['price']}",
        color='purple',
        fill=True
    ).add_to(map)

locations = zip(data_s['latitude'], data_s['longitude'])
HeatMap(locations).add_to(map)

In [ ]:
map

In [ ]:
data_s = data.sample(5000)

In [ ]:
marker_cluster = MarkerCluster(
    name='CPH Airbnb',
    overlay=True,
    control=False,
    icon_create_function=None,
    showCoverageOnHover = False
)

In [ ]:
#Define the map
map = folium.Map(location=[55.7800, 12.5400], zoom_start=12)

#add prettier base-layer
folium.TileLayer('cartodbpositron').add_to(map)

#add points (in a loop)
for j in data_s.iterrows(): # <--- using the whole dataset
    marker= folium.Circle(
        radius=30,
        location=[j[1]['latitude'], j[1]['longitude']],
        tooltip=j[1]['name'],
        popup=f"<b>{j[1]['name']}</b>\n {j[1]['price']}",
        color='purple',
        fill=True
    )
    marker_cluster.add_child(marker) # <<--- here we add the marker to the cluster rather than to the map directly

marker_cluster.add_to(map)

locations = zip(data_s['latitude'], data_s['longitude'])
HeatMap(locations).add_to(map)


In [ ]:
map

In [ ]:
map.save('cph_folium.html')

In [ ]:
# Define a layer to display on a map

layer = pdk.Layer(
    "HexagonLayer",
    data,
    get_position=["longitude", "latitude"],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    radius=50,
    opacity=0.3,
    elevation_range=[0, 200],
    extruded=True,
    coverage=1,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=data['latitude'].mean(), longitude=data['longitude'].mean(), zoom=11, min_zoom=5, max_zoom=15, pitch=40.5, bearing=-27.36)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{elevationValue}"})

In [ ]:
r.to_html(r'airbnb_hexa.html')